# Brain tumor detection using tensorflow and opencv

In [ ]:
# importing libraries

import os
import tensorflow as tf
from keras import layers
from keras import Model
from keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# saving the location of the base directory

base_dir = r"C:\Users\91976\Desktop\programming\AI and Ml\projects\survival predicton\Tumor"

In [ ]:
# storing the location of the training  and testing directories 

test_dir = f'{base_dir}\images\Testing'
train_dir = f"{base_dir}\images\Training"

train_glioma_tumor_dir = f'{train_dir}\glioma_tumor'
train_meningioma_tumor_dir = f'{train_dir}\meningioma_tumor'
train_no_tumor_dir = fr'{train_dir}\no_tumor'
train_pituitary_tumor_dir = f'{train_dir}\pituitary_tumor'


test_glioma_tumor_dir = f'{test_dir}\glioma_tumor'
test_meningioma_tumor_dir = f'{test_dir}\meningioma_tumor'
test_no_tumor_dir = fr"{test_dir}\no_tumor"
test_pituitary_tumor_dir = f'{test_dir}\pituitary_tumor'



In [ ]:





labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']


X_train = []
y_train = []



image_size = 224
for i in labels:
    folderPath = f"{train_dir}\{i}"
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
       
for i in labels:
    folderPath = f"{test_dir}\{i}"
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)


        
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)

In [ ]:
# spittting the data into test and train for validation

X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)


y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
    
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)



print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)



# importing pre trained model InceptionV3

In [ ]:

pre_model = MobileNetV2(
        input_shape=(224,224,3),
        weights='imagenet',
        include_top=False
)       




'''
pre_model = InceptionV3(
        input_shape=(224,224,3),
        weights='imagenet',   
        include_top=False,
            
)
'''

'''
pre_model = VGG16(
        input_shape=(224,224,3),
        weights='imagenet',
        include_top=False,
        pooling='max'
)
'''

# freezing the layers 

for layers in pre_model.layers:
    layers.trainable=False


last_output = pre_model.layers[-2].output   

'''
# using the mixed7 layer for output of inceptionv3

last_layer = pre_model.get_layer('mixed7')
last_output = last_layer.output
'''  

# Model formation and training

In [ ]:
from keras.optimizers import Adam,RMSprop

x = tf.keras.layers.Rescaling(scale=1./255)(last_output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(32,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(16,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(4,activation='softmax')(x)

model = Model(pre_model.input,output)

model.compile(optimizer=Adam(learning_rate=0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy']
)

In [ ]:
# stopping the training as soon as the val_accuracy reaches 0.95


class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>0.92):
            self.model.stop_training = True

training the model

In [ ]:



callback = myCallbacks()


 

model.fit(

    X_train,y_train, batch_size=32,
    epochs=10,
    validation_data=(X_test,y_test),
    steps_per_epoch = len(X_train) / 32 ,
    callbacks=[callback]
)

# Saving the model

In [ ]:
#model.save('tumor_mobilenet.h5')

In [ ]:

'''
tf_model_file = "tumor_mobilenet.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

with open(tf_model_file,'wb') as file:
    file.write(tflite_model)
''' 

Testing

In [ ]:
t = []

img = cv2.imread(r"C:\Users\91976\Desktop\programming\AI and Ml\projects\survival predicton\Tumor\images\Testing\meningioma_tumor\image(1).jpg")

img = cv2.resize(img,(224,224))
t.append(img)

t_p = np.array(t)

In [ ]:

pred = model.predict(t_p)

In [ ]:
print(pred)
predictions = np.argmax(pred,1)


In [ ]:
'''
labels = {

    'glioma_tumor': 0,
    'no_tumor' : 1,
    'meningioma_tumor': 2,
    'pituitary_tumor' : 3


}


'''